In [1]:
import os
import random

import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as nfunc
from torch.nn.parameter import Parameter
from torch.utils.data import TensorDataset, DataLoader

gpu = ""

os.environ["TF_CPP_MIN_LOG_LEVEL"] = '2'
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES'] = gpu

device = torch.device("cuda" if gpu else "cpu")

In [2]:
def call_bn(bn, x, update_batch_stats=True):
    if bn.training is False:
        return bn(x)
    elif not update_batch_stats:
        return nfunc.batch_norm(x, None, None, bn.weight, bn.bias, True, bn.momentum, bn.eps)
    else:
        return bn(x)
    
    
class MLP(nn.Module):
    def __init__(self, layer_sizes, affine=False, top_bn=True):
        super(MLP, self).__init__()
        self.input_len = 1 * 28 * 28
        self.fc1 = nn.Linear(self.input_len, 1200)
        self.fc2 = nn.Linear(1200, 1200)
        self.fc3 = nn.Linear(1200, 10)

        self.bn_fc1 = nn.BatchNorm1d(1200, affine=affine)
        self.bn_fc2 = nn.BatchNorm1d(1200, affine=affine)
        self.top_bn = top_bn
        if top_bn:
            self.bn_fc3 = nn.BatchNorm1d(10, affine=affine)

    def forward(self, x, update_batch_stats=True):
        h = nfunc.relu(self.fc1(x.view(-1, self.input_len)))
        h = self.fc2(h)
        h = self.bn_fc2(h)
        h = nfunc.relu(h)
        h = self.fc3(h)
        logits = h

        return logits

class MLP2(nn.Module):
    def __init__(self, layer_sizes, affine=False, top_bn=True):
        super(MLP2, self).__init__()
        self.input_len = 1 * 28 * 28
        self.fc1 = nn.Linear(self.input_len, 1200)
        self.fc2 = nn.Linear(1200, 1200)
        self.fc3 = nn.Linear(1200, 10)


    def forward(self, x, update_batch_stats=True):
        h = nfunc.relu(self.fc1(x.view(-1, self.input_len)))
        h = self.fc2(h)
        h = nfunc.relu(h)
        h = self.fc3(h)
        logits = h

        return logits

In [3]:
torch.__version__

'1.1.0.dev20190507'

In [12]:
from torch.utils.tensorboard import SummaryWriter
exp_marker = "BoardTest/1"
base_dir = os.path.join(os.environ['HOME'], 'project/runs')
dir_path = os.path.join(base_dir, exp_marker)
if not os.path.isdir(dir_path):
    os.makedirs(dir_path)

writer = SummaryWriter(dir_path)

In [13]:
writer.add_scalar("Test/acc", 10, 1)

In [7]:
model = MLP(None)
model(torch.zeros(2, 1, 28, 28))

model = MLP2(None)
model(torch.zeros(2, 1, 28, 28))

tensor([[-0.0191,  0.0168,  0.0017,  0.0009,  0.0158,  0.0056,  0.0163, -0.0055,
          0.0195,  0.0162],
        [-0.0191,  0.0168,  0.0017,  0.0009,  0.0158,  0.0056,  0.0163, -0.0055,
          0.0195,  0.0162]], grad_fn=<AddmmBackward>)

In [14]:
dummy_input = (torch.zeros(2, 1, 28, 28),)
writer.add_graph(MLP(None), dummy_input, True)

frame #0: std::function<std::string ()>::operator()() const + 0x11 (0x7fe08c3da421 in /home/xyang2/project/other/DLCodeSnippet/venv/lib/python3.6/site-packages/torch/lib/libc10.so)
frame #1: c10::Error::Error(c10::SourceLocation, std::string const&) + 0x2a (0x7fe08c3d9d5a in /home/xyang2/project/other/DLCodeSnippet/venv/lib/python3.6/site-packages/torch/lib/libc10.so)
frame #2: <unknown function> + 0x42e675 (0x7fe084799675 in /home/xyang2/project/other/DLCodeSnippet/venv/lib/python3.6/site-packages/torch/lib/libtorch_python.so)
frame #3: <unknown function> + 0x131dfc (0x7fe08449cdfc in /home/xyang2/project/other/DLCodeSnippet/venv/lib/python3.6/site-packages/torch/lib/libtorch_python.so)
<omitting python frames>
frame #26: <unknown function> + 0x1b0134 (0x7fe08451b134 in /home/xyang2/project/other/DLCodeSnippet/venv/lib/python3.6/site-packages/torch/lib/libtorch_python.so)
frame #27: torch::jit::BlockToONNX(torch::jit::Block*, torch::jit::Block*, torch::onnx::OperatorExportTypes, std::

graph(%0 : Float(2, 1, 28, 28),
      %1 : Float(1200, 784),
      %2 : Float(1200),
      %3 : Float(1200, 1200),
      %4 : Float(1200),
      %5 : Float(10, 1200),
      %6 : Float(10),
      %7 : Float(1200),
      %8 : Float(1200),
      %9 : Long(),
      %10 : Float(1200),
      %11 : Float(1200),
      %12 : Long(),
      %13 : Float(10),
      %14 : Float(10),
      %15 : Long()):
  %57 : int[] = prim::Constant[value=[-1, 784]]()
  %input.1 : Float(2, 784) = aten::view(%0, %57), scope: MLP
  %36 : Float(784!, 1200!) = aten::t(%1), scope: MLP/Linear[fc1]
  %58 : Long() = prim::Constant[value={1}](), scope: MLP/Linear[fc1]
  %59 : Long() = prim::Constant[value={1}](), scope: MLP/Linear[fc1]
  %39 : Float(2, 1200) = aten::addmm(%2, %input.1, %36, %58, %59), scope: MLP/Linear[fc1]
  %input.2 : Float(2, 1200) = aten::relu(%39), scope: MLP
  %41 : Float(1200!, 1200!) = aten::t(%3), scope: MLP/Linear[fc2]
  %60 : Long() = prim::Constant[value={1}](), scope: MLP/Linear[fc2]
  %61 : Lo

In [9]:
print(torch.__version__)
dummy_input = (torch.zeros(2, 1, 28, 28),)
writer.add_graph(MLP2(None), dummy_input, True)

1.1.0.dev20190507
graph(%0 : Float(2, 1, 28, 28),
      %1 : Float(1200, 784),
      %2 : Float(1200),
      %3 : Float(1200, 1200),
      %4 : Float(1200),
      %5 : Float(10, 1200),
      %6 : Float(10)):
  %7 : Tensor = onnx::Constant[value=  -1  784 [ Variable[CPUType]{2} ]](), scope: MLP2
  %8 : Float(2, 784) = onnx::Reshape(%0, %7), scope: MLP2
  %9 : Float(784!, 1200!) = onnx::Transpose[perm=[1, 0]](%1), scope: MLP2/Linear[fc1]
  %10 : Float(2, 1200) = onnx::Gemm[alpha=1, beta=1](%8, %9, %2), scope: MLP2/Linear[fc1]
  %11 : Float(2, 1200) = onnx::Relu(%10), scope: MLP2
  %12 : Float(1200!, 1200!) = onnx::Transpose[perm=[1, 0]](%3), scope: MLP2/Linear[fc2]
  %13 : Float(2, 1200) = onnx::Gemm[alpha=1, beta=1](%11, %12, %4), scope: MLP2/Linear[fc2]
  %14 : Float(2, 1200) = onnx::Relu(%13), scope: MLP2
  %15 : Float(1200!, 10!) = onnx::Transpose[perm=[1, 0]](%5), scope: MLP2/Linear[fc3]
  %16 : Float(2, 10) = onnx::Gemm[alpha=1, beta=1](%14, %15, %6), scope: MLP2/Linear[fc3]
  retu

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.autograd import Variable


dummy_input = (torch.zeros(1, 3),)

class LinearInLinear(nn.Module):
    def __init__(self):
        super(LinearInLinear, self).__init__()
        self.l = nn.Linear(3, 5)

    def forward(self, x):
        return self.l(x)
    
writer.add_graph(LinearInLinear(), dummy_input, verbose=True)

graph(%input : Float(1, 3),
      %1 : Float(5, 3),
      %2 : Float(5)):
  %3 : Float(3!, 5!) = onnx::Transpose[perm=[1, 0]](%1), scope: LinearInLinear/Linear[l]
  %4 : Float(1, 5) = onnx::Gemm[alpha=1, beta=1](%input, %3, %2), scope: LinearInLinear/Linear[l]
  return (%4)



In [21]:
torch.__version__

'1.0.1.post2'